In [89]:
import yfinance as yf
yf.pdr_override()
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import statsmodels.api as sm

CODES = ['042700.KS', 'NVDA'] # 원하는 기업 데이터
for CODE in CODES:
    
    if CODE.endswith('.KS'): # 한국 기업이면 코스피 사용
        KOSPI_CODE = '^KS11'
        RF = 3.119
        ERP = 5.36
    else:
        KOSPI_CODE = '^IXIC' # 다른 나라 기업이면 일단은 미국 기업으로 산정
        RF = 2.8741
        ERP = 4.24

    FIRST = '2017-05-08'
    TODAY = '2022-05-06' #dt.strftime(dt.today(),"%Y-%m-%d")
    INTERVAL ='1wk'
    
    price = yf.download(CODE, start = FIRST, end = TODAY, interval=INTERVAL, progress=False)
    kospi = yf.download(KOSPI_CODE, start = FIRST, end = TODAY, interval=INTERVAL, progress=False)

    df_stock = pd.concat([price['Close'], round(price['Close'].pct_change()*100,2)], axis=1)
    df_kospi = pd.concat([kospi['Close'], round(kospi['Close'].pct_change()*100,2)], axis=1)

    df_stock.columns = ['stock_price', 'stock_pct']
    df_stock = df_stock.dropna()

    df_kospi.columns = ['kospi_price', 'kospi_pct']
    df_kospi = df_kospi.dropna()

    df = pd.concat([df_kospi, df_stock], axis=1, join='inner',)

    y = df['stock_pct'].values
    X = df['kospi_pct'].values

    X = X.reshape(-1,1)
    y = y.reshape(-1,1)

    results = sm.OLS(y, sm.add_constant(X)).fit()
    BETA = results.params[1]

    print('--------',CODE,'---------')
    print('Rf :',RF)
    print('ERP :', ERP)
    print("Beta :", BETA)

    rm = RF + BETA*ERP
    print('E(r) :',rm)
    print('')

-------- 042700.KS ---------
Rf : 3.119
ERP : 5.36
Beta : 1.1513007916091724
E(r) : 9.289972243025165

-------- NVDA ---------
Rf : 2.8741
ERP : 4.24
Beta : 1.6197383500870792
E(r) : 9.741790604369216

